In [5]:
########## CONFIGURATION ###########
GRAPH_PATH = "../out/graph.ttl"

In [6]:
import json
from glob import glob

data_authors = []
for filename in glob("..\\data_collection\\1_get_author_ids_output\\*.json"):
    with open(filename, encoding="utf8") as f:
        data_authors.extend(json.load(f))

data_authors[:2]

[{'search_name': 'Christian Ahlf',
  'institution_id': 'https://openalex.org/I70451448',
  'results': []},
 {'search_name': 'Jakob Andersen',
  'institution_id': 'https://openalex.org/I70451448',
  'results': [{'id': 'https://openalex.org/A2525626150',
    'display_name': 'Jakob Smedegaard Andersen'},
   {'id': 'https://openalex.org/A4214226477',
    'display_name': 'Jakob Andersen'}]}]

In [7]:
data_works = []
for filename in glob("..\\data_collection\\2_get_works_output\\*.json"):
    with open(filename, encoding="utf8") as f:
        data_works.extend(json.load(f))

data_works[0]

{'id': 'https://openalex.org/W3176618563',
 'doi': 'https://doi.org/10.18653/v1/2021.acl-demo.17',
 'title': 'REM: Efficient Semi-Automated Real-Time Moderation of Online Forums',
 'display_name': 'REM: Efficient Semi-Automated Real-Time Moderation of Online Forums',
 'publication_year': 2021,
 'publication_date': '2021-08-01',
 'ids': {'openalex': 'https://openalex.org/W3176618563',
  'doi': 'https://doi.org/10.18653/v1/2021.acl-demo.17',
  'mag': '3176618563'},
 'host_venue': {'id': None,
  'issn_l': None,
  'issn': None,
  'display_name': 'meeting of the association for computational linguistics',
  'publisher': 'Association for Computational Linguistics',
  'type': 'publisher',
  'url': 'https://doi.org/10.18653/v1/2021.acl-demo.17',
  'is_oa': True,
  'version': 'publishedVersion',
  'license': 'cc-by'},
 'type': 'proceedings-article',
 'open_access': {'is_oa': True,
  'oa_status': 'hybrid',
  'oa_url': 'https://aclanthology.org/2021.acl-demo.17.pdf'},
 'authorships': [{'author_po

In [8]:


from oa_graph_json import  OpenAlexGraph, _resource_from_uri

g = OpenAlexGraph()

for item in data_works:
    host_venue = item["host_venue"]
    host_id = _resource_from_uri(host_venue["id"])

    # Add work
    work_id = _resource_from_uri(item["id"])
    g.add_work(
        identifier=work_id,
        name=item["display_name"],
        publisher=host_id,
        publish_date=item["publication_date"]
    )

    for cites_by_year in item["counts_by_year"]:
        g.add_citations_in_year(
            identifier=work_id,
            year=cites_by_year["year"],
            citations=cites_by_year["cited_by_count"]
        )

    # Add authors of paper and is_author relation
    for authorship in item["authorships"]:
        author = authorship["author"]
        author_id = _resource_from_uri(author["id"])
        author_name = author["display_name"]

        inst_id = None
        if authorship["institutions"]:
            inst = authorship["institutions"][0]
            inst_id = _resource_from_uri(inst["id"])
            inst_name = inst["display_name"]
            g.add_institution(inst_id, inst_name)
            g.add_associated_with_institution(work_id, inst_id)

        g.add_author(author_id, author_name, inst_id)

        g.add_is_author(author_id, work_id)

KeyError: 'geo'

In [ ]:
g.serialize(GRAPH_PATH)

In [ ]:
len(g)